In [ ]:
import os
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
from datetime import datetime


In [ ]:
# Set up the YouTube API client
credentials, _ = google.auth.default(scopes=['https://www.googleapis.com/auth/youtube.upload'])
youtube = build('youtube', 'v3', credentials=credentials)


In [ ]:

# Read the Excel sheet
df = pd.read_excel('/content/youtube.xlsx')

# Loop through the DataFrame rows and upload the videos/posts
for index, row in df.iterrows():
    # Extract details from the row
    title = row['title']
    description = row['description']
    filepath = row['filepath']
    platform = row['platform']
    datetime = row['datetime']
    tags = row['tags'].split(',')
    privacy = row['privacy']
    comments = row['comments']
    # Convert date_time to ISO 8601 format
    #iso_date_time = datetime.strftime(datetime, '%Y-%m-%dT%H:%M:%S.%fZ')


    # Upload the video/post to the specified platform
    if platform.lower() == 'youtube':
        try:
            # Set video metadata
            video_metadata = {
                'snippet': {
                    'title': title,
                    'description': description,
                    'tags': tags
                },
                'status': {
                    'privacyStatus': 'public',
                    'publishAt': datetime
                }
            }

            # Upload the video to the channel
            response = youtube.videos().insert(part='snippet,status', body=video_metadata, media_body=filepath).execute()

            # Update the Excel sheet with the video URL
            df.at[index, 'Video URL'] = f'https://www.youtube.com/watch?v={response["id"]}'
        except HttpError as e:
            print(f'An error occurred: {e}')
    elif platform.lower() == 'facebook':
        try:
            # Set video metadata
            video_metadata = {
                'title': title,
                'description': description,
                'tags': tags.split(',') if tags else []
            }

            # Upload the video to the Facebook page
            response = graph.put_video(file=open(filepath, 'rb'), title=title, description=description, tags=tags.split(','))

            # Update the Excel sheet with the video URL
            df.at[index, 'Video URL'] = f'https://www.facebook.com/{response["id"]}'
        except Exception as e:
            print(f'An error occurred: {e}')



TypeError: ignored

In [ ]:

# Save the updated Excel sheet
df.to_excel('schedule-updated.xlsx', index=False)

In [ ]:
# Upload the video to YouTube
def upload_video(youtube, video_metadata, video_file):
    insert_request = youtube.videos().insert(
        part=','.join(video_metadata.keys()),
        body=video_metadata,
        media_body=video_file
    )
    response = insert_request.execute()
    print(f'Video uploaded: {response["id"]}')



In [ ]:
# Read the Excel sheet and upload the videos
def upload_videos_from_excel(filename):
    df = pd.read_excel(filename)
    youtube = get_authenticated_service()
    for i, row in df.iterrows():
        video_metadata, video_file = get_video_metadata(row)
        upload_video(youtube, video_metadata, video_file)



In [ ]:
if __name__ == '__main__':
    filename = 'videos.xlsx' # replace with your Excel file name
    upload_videos_from_excel(filename)
